Elliot Ide Pozo

- <del>Crear script en python que permita extraer la data desde el server de base de datos (activismo.inf.uach.cl)</del>
- <del>Crear script en python que permita crear un json (dataset) de desafío, usuario, consulta, resultado (diccionario).</del>
- <del>Crear un script para obtener la representación tf-idf (NLTK,,,).</del>
- Crear un script que calcule al menos una métrica de las establecidas por cada conjunto.


In [1]:
from sshtunnel import SSHTunnelForwarder
import pymysql
import pandas as pd

In [2]:
# https://stackoverflow.com/questions/21903411/enable-python-to-connect-to-mysql-via-ssh-tunnelling
# https://stackoverflow.com/questions/42726681/conecting-to-mysql-in-a-remote-server-from-python

with SSHTunnelForwarder(
            'activismo.inf.uach.cl',
            ssh_username='vps238',
            ssh_password='vps238.2018',
            remote_bind_address=('127.0.0.1', 3306)) as tunnel:
        conn = pymysql.connect(host='127.0.0.1', user='learning',
                               passwd='analytic$',db='gonsa2',
                               port=tunnel.local_bind_port)
        query = "SELECT date_executed, FK_challenge_id_number, FK_student_nick,id_number,query,snippet \
        FROM Students_query\
        INNER JOIN Students_queries_results ON FK_student_query_id_number = id_number\
        WHERE FK_challenge_id_number = 1 AND FK_student_nick != 'test_student'\
        LIMIT 20";
        data = pd.read_sql(query, conn)
        conn.close()

In [3]:
from bs4 import BeautifulSoup

In [4]:
#https://www.crummy.com/software/BeautifulSoup/bs4/doc/
#https://stackoverflow.com/questions/19798153/difference-between-map-applymap-and-apply-methods-in-pandas
    
# transformar en un objeto BeautifulSoup, para luego retornar el texto sin tags
data['snippet'] = data['snippet'].map(lambda text: BeautifulSoup(text, 'html.parser').get_text())

In [5]:
data

,date_executed,FK_challenge_id_number,FK_student_nick,id_number,query,snippet
0,2019-04-29 14:39:53,1,eespinoza,11,vih en chile,vih en chile. Es urgente aumentar el diagnósti...
1,2019-04-29 14:39:53,1,eespinoza,11,vih en chile,"En Chile, la Ley 19.779 resguarda los derechos..."
2,2019-04-29 14:39:53,1,eespinoza,11,vih en chile,¿Cuánto tarda una persona con VIH en llegar a ...
3,2019-04-29 14:39:53,1,eespinoza,11,vih en chile,VIH Sida en Chile: Cifras del Minsal indican q...
4,2019-04-29 14:39:53,1,eespinoza,11,vih en chile,En los últimos diez años los contagios de VIH ...
5,2019-04-29 14:39:53,1,eespinoza,11,vih en chile,Advierten que el VIH sida está descontrolado e...
6,2019-04-29 14:39:53,1,eespinoza,11,vih en chile,SIDA en Chile: la epidemia que crece silencios...
7,2019-04-29 14:39:53,1,eespinoza,11,vih en chile,del tratamiento anti-vih para ayudar y protege...
8,2019-04-29 14:39:53,1,eespinoza,11,vih en chile,Para el presidente de la Corporación Sida Chil...
9,2019-04-29 14:39:53,1,eespinoza,11,vih en chile,Informe revela que Chile es el país donde más ...


### Estimación TF-IDF 

Fuente: 

https://www.youtube.com/watch?v=hc3DCn8viWs
    
**Drawbacks**
- Only based on terms
- Weak on capturing document topic
- Weak handling synonyms

**How to overcome drawbacks**
- LSA (Latent Semantic Analysis)
- Word Embedding (Word2Vec, Glove)
- ConceptNet

In [6]:
q = pd.Series(data['query'][0])

In [7]:
corpus = q.append(data['snippet'][:3], ignore_index=True) # q, d1, d2 y d3
corpus

0                                         vih en chile
1    vih en chile. Es urgente aumentar el diagnósti...
2    En Chile, la Ley 19.779 resguarda los derechos...
3    ¿Cuánto tarda una persona con VIH en llegar a ...
dtype: object

In [8]:
import nltk
from nltk.corpus import stopwords

es_sw = stopwords.words('spanish') # lista stopword a usar

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
tf = TfidfVectorizer(stop_words=es_sw) # crear la transformación

tfidf_matrix = tf.fit_transform(corpus)

In [11]:
# https://stackoverflow.com/questions/34449127/sklearn-tfidf-transformer-how-to-get-tf-idf-values-of-given-words-in-documen

feature_names = tf.get_feature_names()
print(feature_names, len(feature_names))

['19', '2016', '2018', '779', 'acceso', 'adecuadamente', 'asimismo', 'aumentar', 'calcula', 'cautela', 'chile', 'contagia', 'control', 'corporación', 'cuánto', 'cómo', 'derechos', 'desarrollar', 'diagnosticada', 'diagnóstico', 'discriminaciones', 'educación', 'ejercicio', 'estudiada', 'etapa', 'igualitario', 'impidiendo', 'infección', 'infectada', 'infectadas', 'infectarse', 'latinoamérica', 'ley', 'libre', 'llegar', 'mitad', 'periodo', 'persona', 'personas', 'prevalencia', 'prevención', 'producen', 'puedo', 'resguarda', 'respecto', 'saber', 'salud', 'si', 'sida', 'sido', 'síntomas', 'tarda', 'trabajo', 'tratamiento', 'urgente', 'ventana', 'vih', 'viven'] 58


In [12]:
#https://towardsdatascience.com/natural-language-processing-feature-engineering-using-tf-idf-e8b9d00e7e76

dense = tfidf_matrix.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)
df

,19,2016,2018,779,acceso,adecuadamente,asimismo,aumentar,calcula,cautela,...,sida,sido,síntomas,tarda,trabajo,tratamiento,urgente,ventana,vih,viven
0,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000,0.707107,0.000000
1,0.000000,0.00000,0.00000,0.000000,0.000000,0.216501,0.000000,0.216501,0.216501,0.000000,...,0.000000,0.433003,0.00000,0.00000,0.000000,0.170692,0.216501,0.00000,0.112979,0.000000
2,0.182708,0.00000,0.00000,0.182708,0.182708,0.000000,0.182708,0.000000,0.000000,0.182708,...,0.144049,0.000000,0.00000,0.00000,0.182708,0.144049,0.000000,0.00000,0.190690,0.182708
3,0.000000,0.15728,0.15728,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.372003,0.000000,0.15728,0.15728,0.000000,0.000000,0.000000,0.15728,0.246225,0.000000


In [13]:
print(tfidf_matrix[1])

  (0, 53)	0.17069209050639012
  (0, 12)	0.17069209050639012
  (0, 18)	0.2165012731424809
  (0, 28)	0.17069209050639012
  (0, 35)	0.2165012731424809
  (0, 8)	0.2165012731424809
  (0, 23)	0.2165012731424809
  (0, 5)	0.2165012731424809
  (0, 49)	0.4330025462849618
  (0, 31)	0.2165012731424809
  (0, 27)	0.2165012731424809
  (0, 39)	0.2165012731424809
  (0, 29)	0.2165012731424809
  (0, 38)	0.34138418101278023
  (0, 19)	0.17069209050639012
  (0, 7)	0.2165012731424809
  (0, 54)	0.2165012731424809
  (0, 10)	0.22595869148805003
  (0, 56)	0.11297934574402502


### Similaridad del coseno

Fuentes:

https://intellipaat.com/community/1103/python-tf-idf-cosine-to-find-document-similarity

In [14]:
from sklearn.metrics.pairwise import linear_kernel
import numpy as np

In [15]:
S = []
for i in range(corpus.count()):
    S.append(linear_kernel(tfidf_matrix[i], tfidf_matrix).flatten()) # equivalente a cos

S = np.asarray(S)
S

array([[1.        , 0.23966538, 0.20225696, 0.23214321],
       [0.23966538, 1.        , 0.21520469, 0.06752991],
       [0.20225696, 0.21520469, 1.        , 0.14408941],
       [0.23214321, 0.06752991, 0.14408941, 1.        ]])

### Estimacion MMR

Fuentes:

http://www.cs.bilkent.edu.tr/~canf/CS533/hwSpring14/eightMinPresentations/handoutMMR.pdf

In [16]:
#https://www.quora.com/Where-can-I-find-a-maximum-marginal-relevance-algorithm-in-Python-for-redundancy-removal-in-two-documents
    
def mmr_sorted(docs, q, lambda_, similarity1, similarity2):
    """Sort a list of docs by Maximal marginal relevance
	
	Performs maximal marginal relevance sorting on a set of
	documents as described by Carbonell and Goldstein (1998)
	in their paper "The Use of MMR, Diversity-Based Reranking
	for Reordering Documents and Producing Summaries"
 
    :param docs: a set of documents to be ranked 
				  by maximal marginal relevance
    :param q: query to which the documents are results
    :param lambda_: lambda parameter, a float between 0 and 1
    :param similarity1: sim_1 function. takes a doc and the query
						as an argument and computes their similarity
    :param similarity2: sim_2 function. takes two docs as arguments
						and computes their similarity score
    :return: a (document, mmr score) ordered dictionary of the docs
			given in the first argument, ordered my MMR
    """
    selected = collections.OrderedDict()
    while set(selected) != docs:
        remaining = docs - set(selected)
        mmr_score = lambda x: lambda_*similarity1(x, q) - (1-lambda_)*max([similarity2(x, y) for y in set(selected)-{x}] or [0])
        next_selected = argmax(remaining, mmr_score)
        selected[next_selected] = len(selected)
    return selected
 
def argmax(keys, f):
    return max(keys, key=f)

In [17]:
S

array([[1.        , 0.23966538, 0.20225696, 0.23214321],
       [0.23966538, 1.        , 0.21520469, 0.06752991],
       [0.20225696, 0.21520469, 1.        , 0.14408941],
       [0.23214321, 0.06752991, 0.14408941, 1.        ]])

In [18]:
S[1,0]

0.23966538452885877

In [19]:
docs = list(range(1,corpus.count()))
init_idx= np.argmax(S[0][1::]) + 1 # el indice del d con mayor similaridad a q
selected = []
#selected.append(init_idx.item())

In [20]:
docs, selected

([1, 2, 3], [])

In [21]:
#while selected != docs:
#remaining = [i for i in docs if i not in selected]
#mmr_score = lambda x: (lambda_*S[0,x] -(1 - lambda_)* max(S[ele, x])) for x in remaining

In [22]:
def argmax(keys, f):
    return max(keys, key=f)

In [23]:
#while selected != docs:
#remaining = [i for i in docs if i not in selected]
#mmr_score = lambda x: lambda_*S[x, 0] - (1-lambda_)*max([S[x, y] for y in set(selected)-{x}] or [0])
#next_selected = argmax(remaining, mmr_score)
#selected[next_selected] = len(selected)